In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from tqdm import tqdm
tqdm.pandas(desc="Applying function")

from shapely.geometry import Point

In [2]:
gdf_buildings = gpd.read_file('../../data/processed/building/building_sample.geojson')

In [3]:
gdf_buildings.head()

,height,mpluto_bbl,cnstrct_yr,globalid,bin,shape_len,shape_area,office_area,retail_area,residential_area,...,idw_aadt_2019,idw_aadt_2020,idw_aadt_2021,idw_atvc_2018,idw_atvc_2019,idw_atvc_2020,office_within_450ft,retail_within_450ft,residential_within_450ft,geometry
0,55.89,1006100054,1890,{3AEADBE0-CC54-4D24-B238-035EACC2FCA7},1010689,0.0,636520502.758,0,1050,8294,...,7193.081669,7412.369895,8350.144642,123.350334,147.058962,304.025386,2120,52008,312275,"MULTIPOLYGON (((984248.206 206864.602, 984228...."
1,43.48,1014390003,1920,{16E3A900-8D55-4377-ABC9-3CFEAD5AABEA},1044690,None,636520502.758,0,2175,6525,...,12988.853950,12048.318975,13486.718605,151.540038,189.112186,239.713791,9539,49216,1197236,POINT (994870.595 217574.392)
2,164.86,1011480001,1925,{A569DF64-C060-4E22-A45B-C3AAF3994CF0},1030169,0.0,636520502.758,0,5000,118840,...,13677.792850,13732.269067,15411.109097,126.416875,155.399025,245.756263,2205,52578,438942,"MULTIPOLYGON (((989996.502 223961.988, 989991...."
3,57.41,1019660033,1901,{FF1D608A-A8F9-4BED-8330-01D6E785BBDE},1084099,0.0,636520502.758,0,3500,15130,...,11736.070949,12900.752001,14449.074278,100.988607,140.355271,284.405716,35299,23925,795466,"MULTIPOLYGON (((996485.007 235709.202, 996487...."
4,80.0,1019880018,2017,{F3EEBE92-794C-4F0B-914A-86A784D1532B},1089415,0.0,636520502.758,2962,0,16905,...,12835.701327,14143.073155,15793.074614,88.857910,146.777034,285.662061,2962,23684,68622,"MULTIPOLYGON (((996978.357 237617.591, 997012...."


In [4]:
gdf_buildings.crs

<Projected CRS: EPSG:2263>
Name: NAD83 / New York Long Island (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - New York - counties of Bronx; Kings; Nassau; New York; Queens; Richmond; Suffolk.
- bounds: (-74.26, 40.47, -71.8, 41.3)
Coordinate Operation:
- name: SPCS83 New York Long Island zone (US survey foot)
- method: Lambert Conic Conformal (2SP)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

## Distance from Park

In [5]:
gdf_parks = gpd.read_file('../../data/raw/Parks Properties_20240405.geojson').to_crs(gdf_buildings.crs)

In [6]:
gdf_buildings = gdf_buildings.sjoin_nearest(gdf_parks.loc[:,['geometry']],distance_col='distance_to_park').drop(columns='index_right')

## Distance from School

In [7]:
df_school = pd.read_csv('../../data/raw/2019_-_2020_School_Locations_20240405.csv')

In [8]:
points = [Point(x,y) for x,y in zip(df_school.loc[:, 'LONGITUDE'], df_school.loc[:, 'LATITUDE'])]
gdf_school = gpd.GeoDataFrame(df_school, geometry=points, crs=4326)
gdf_school = gdf_school.to_crs(gdf_buildings.crs)

In [9]:
gdf_buildings = gdf_buildings.sjoin_nearest(gdf_school.loc[:,['geometry']],distance_col='distance_to_school').drop(columns='index_right')

In [10]:
gdf_buildings = gdf_buildings.drop_duplicates(subset=['bin'])

In [11]:
gdf_buildings.to_file('../../data/processed/building/building_sample.geojson', driver='GeoJSON')